# 交差検証とグリッドサーチ
## 前回行ったこと
 - one-hotエンコーディング
 - 検証用データを用いたモデルの投稿前の評価
 - （付録）欠損値処理

## 今回行うこと

- 交差検証とハイパーパラメータチューニングのためのクラス`GridSearchCV`の使い方
- 分類モデル（サポートベクターマシン）による分類
- 学習後のモデルの観察

## （復習）データの読み込み・特徴ベクトルの構築
前回の資料で構成した，one-hotエンコーディングを用いた特徴ベクトルを再び作ります．
もう詳しく説明することはしません．
詳細は前回の資料を参照してください．

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
# Google Colab を用いている場合，このセルを実行
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colab を用いている場合，このセルを実行
d_train = pd.read_csv("drive/My Drive/data/train.csv")
d_test = pd.read_csv("drive/My Drive/data/test.csv")

In [ ]:
# 自身のPython環境で動かしている場合，このセルを実行（パスは適宜変更）
#d_train = pd.read_csv("data/train.csv")
#d_test = pd.read_csv("data/test.csv")

In [ ]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")

# targetの値
y_train = d_train.pop('stroke')
y_train = y_train.values # numpyのarrayに変換
print(y_train)

以下のセルでone-hotエンコーディングを行い特徴ベクトルを作ります．
前回のQuizの答えです．
変数に`_onehot`とつけるのは面倒なので，今回は`X_train`，`X_test`という変数名にします．

In [ ]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["gender","ever_married", "work_type", "Residence_type", "smoking_status"] # カテゴリカル変数の列名

d_train_test = pd.get_dummies(d_train_test, columns=columns_cat) # get_dummiesを使ってone-hotエンコーディング
d_train_test.pop("bmi") # 今回はbmiデータを使わず，捨てる
d_train = d_train_test[:n_train] # d_train_testの訓練データ部分
d_test = d_train_test[n_train:] # d_train_testのテストデータ部分
X_train = d_train.values # np.arrayに変換
X_test = d_test.values  # np.arrayに変換


## 交差検証（Cross Validation）
特徴ベクトルの作り方やどの手法を使うか，さらには同じ手法であっても**ハイパーパラメータ**（バイアス項を使うか？正則化項の強さはどれくらいか？中間層のユニット数はどれくらいか？等の，ユーザが定める要素）の値によって，予測の正確さは大きく変わってきます．
前回は「ラベル付きデータを訓練用/検証用に分割し，訓練用データで学習した後，学習に使わなかった検証用で評価」ということを行いました．
しかし，訓練・検証の分割の仕方によって評価の値は当然変わってきます．
分割によっては，本当は悪い手法であるのにもかかわらず良い評価となったり，また反対に，本当は良い手法であるのにもかかわらず悪い評価となることもあるでしょう．

そこで，次のような方法がしばしば行われます：データを`K`分割（`K=5,10`等）し，そのうちの`K-1`個の分割を訓練，`1`個の分割を検証データとして用いることを繰り返し，あるハイパーパラメータの時のモデルの性能を`K`回のスコア（評価値）の平均で評価する．
この方法を
**K-分割交差検証（K-fold cross validation）**，あるいは単に**交差検証（cross validation）**
といいます．
K=訓練データ数のときは特にLeave-one-out交差検証（LOOCV）と呼ばれたりもします．
1回だけでなくK回の評価の平均を取っているので，前回の方法よりもより正確に性能の評価ができそうです．
一方で，学習をK回行うため，前回の方法と比べると時間はかかってしまいそうです．

今回はsklearnの便利な関数を使って，いくつかの方法で交差検証を行ってみます．

## ロジスティック回帰の交差検証+グリッドサーチによるハイパーパラメータチューニング

今，使う手法をロジスティック回帰とし，特徴ベクトルは固定して，適切なハイパーパラメータを定めるということを考えます．


In [ ]:
from sklearn.linear_model import LogisticRegression # LogisticRegressionを使えるようにする
lr = LogisticRegression(class_weight='balanced') # インスタンスの作成
lr.fit(X_train, y_train)
y_pred_test_lr = lr.predict_proba(X_test)[:,1]
print(y_pred_test_lr)

[LogisticRegressionのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を見てみましょう．
`fit_intercept`はバイアス項の有無を決めます．
`C`は正則化項の強さを決めるハイパーパラメータです．
ロジスティック回帰においてパラメータ（重み）を定める方法（学習アルゴリズム）はいくつかあり，それは`solver`引数で指定することができます．
`max_iter`や`tol`，`random_state`は`solver`に関係する引数です．
本来はデータの数や次元数などを考慮して適切に選ぶべき（学習時間に大きく影響します）ですが，4つの引数は一旦おいておきます．

練習がてら，`C=10.0`で動かしてみましょう．

In [ ]:
lr = LogisticRegression(C=10.0, class_weight='balanced')
lr.fit(X_train, y_train)
y_pred_test_lr = lr.predict_proba(X_test)[:,1]
print(y_pred_test_lr)

### ロジスティック回帰のハイパーパラメータチューニング：`C`のみチューニング

より小さな`C`を設定すると正則化の効果が強くなり，より重みのノルムが小さくなります．
`C`を非常に小さな値にした場合，重みのノルムはほとんど`0`に近い値となり，訓練データに対する性能もテストデータに対する性能も差はないがどちらも非常に低い，過少適合（アンダーフィッティング，under fitting）が起こってしまいます．
一方で，`C`の値が大きすぎると正則化の効果がほとんどなくなってしまいます．
そのため，`C`は大きすぎても小さすぎてもだめで，いい感じの値を定める必要があります．

ようやく本題です．そこで，**交差検証によって適切な`C`の値を定める**ことにします．
`C`は理論上は任意の非負の値をとることができますが，コンピュータ上ではそれは不可能です．
一般に，連続値を取るハイパーパラメータは，適当に上限と下限と間隔を定め，その中で良い値を探します．
例えば，`[0.1, 0.2, 0.3, 0.4, 0.5]`であったり，`[1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2](=[0.001, 0.01, 0.1, 1, 10, 100])`と言ったようにです．
とりあえず今回は`[1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]`の範囲で探してみましょう．

さらに，交差検証を行う場合，何分割するかを考える必要があります．
とりあえずここでは分割数`K=5`としておきましょう．


これにて行うことが定まりました．以下のようになりそうです：
  1. 訓練データを5分割し，
  2. ある分割を検証用，その他を訓練用として，`C`をある範囲内（ここでは`[1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]`）で動かして学習・評価することを繰り返し，
  3. ある範囲内の全ての値で学習・評価が終わった後，その結果を元に最も良い`C`を定める．

では残りはコードを書くだけなのですが，それなりに面倒なように感じます（もちろん，「書け」と言われれば書けるとは思いますが）．
勉強のために自分で書くことは非常に大事ですが（実験では皆さんに書いてもらうはずです），ここでは偉大なるsklearnの便利な機能を利用します．
sklearnには，[cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)という関数があり，これを使うと簡単に交差検証ができます．
`cross_validate`関数を使う場合，以下の引数を指定する必要があります：
- `estimator`：交差検証によって評価をしたいインスタンス（例：`LogisticRegression`）
- `X`：訓練データ（例：`X_train`）
- `y`：訓練データの目標値（例：`y_train`）
- `cv`：分割数，整数型（例：5）
- `scoring`：評価関数，文字列（例：`"auc_roc"`）

そして以下の情報（キー）を持った**辞書型オブジェクト（=ハッシュテーブル）**を返します：
- `fit_time`：学習にかかった時間のリスト（要素数K）
- `score_time`：評価にかかった時間のリスト（要素数K）
- `test_score`：検証用データでのスコアのリスト（要素数K）

詳しくは上のドキュメントを読んでみてください（実際はここで述べているよりも更に柔軟です）．
評価関数は今回のコンペでは二乗誤差で，sklearnには`roc_auc`で実装されていました．
したがって，これを指定します．

とりあえず使ってみましょう．
`C=10.0`の`LogisticRegression`を検証してみます．

In [ ]:
from sklearn.model_selection import cross_validate
lr = LogisticRegression(C=10.0, class_weight='balanced')
scores = cross_validate(lr, X_train, y_train, cv=5, 
                        scoring="roc_auc")
print(scores)

色々`print`されました．
ちょっと見にくいですね．
今は検証用データのスコアさえあれば良いので，`'test_score'`だけ取り出します．
また，5回の評価結果の平均を計算します．
`np.mean`によってリストや`np.array`に含まれる値の平均を計算できます．
また，符号が反転しているので，わかりやすさのために戻しておきましょう．

In [ ]:
print(f"交差検証の5回のスコア：{scores['test_score']}")
print(f"交差検証の平均スコア：{np.mean(scores['test_score'])}")

結果が出てきました．

では，次は`[1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]`の範囲で`C`の値を変えながら交差検証を行って，最も良い`C`を定めましょう．

### Quiz
上の結果を参考に，`alpha`を`[1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]`の範囲で動かして5分割交差検証を行い，その結果において最も良い（最もスコアが高い）`C`とその交差検証のスコア（5回の検証スコアの平均）を`print`してください．
ここで，`c_list`は探索範囲のリストです．
`results`は，（想定解答の一つにおいて）それぞれの`C`における交差検証のスコアを格納するリストです．
どちらも使っても使わなくても構いません．

楽に書くためのヒント
- リストの末尾に値を追加するのは`append`で行うことができます．
- リスト内の最も大きい値は`np.max`で求めることができます．
- リスト内の最も大きい値のインデックス（場所）は`np.argmax`で求めることができます．

In [ ]:
c_list = [10**i for i in range(-3,3)]
results = []
# ここに何らかの処理を書く
result_best = # ここを埋める
c_best = # ここを埋める
print(f"最良のC：{c_best}　交差検証スコア：{result_best}")

結果が出てきたでしょうか？
私の環境では，`C=0.01`が（この範囲では）最もよく，またその時のスコアは（およそ）`0.77114`となりました．

### ロジスティック回帰のハイパーパラメータチューニング：`C`と`solver`のチューニング

今度は`C`だけでなく`solver`もチューニングしてみましょう．
チューニングするハイパーパラメータが複数ある場合によく行われる方法は，全てのパラメータについて探索範囲を定め，その範囲の全ての組合せを試して（=範囲の直積上で探索して），その中で最も良いハイパーパラメータの組合せを探す，というものです．
これを**グリッドサーチ**といいます．
例えば，`C`を`[1, 10, 100]`，`solver`を`['newton-cg', 'lbfgs', 'sag']`の範囲で探索する場合，
 - `C=1`，`solver='newton-cg'`
 - `C=1`，`solver='lbfgs'`
 - `C=1`，`solver='sag'`
 - `C=10`，`solver='newton-cg'`
 - …
 - `C=100`，`solver='sag'`
 
の合計`3*3=9`通りを試します．


では実際に行ってみようと思います……が，はっきり言ってやや面倒に感じます．
なんと素晴らしいことに，sklearnにはこれを簡単に行う`GridSearchCV`というクラスが提供されています．
`GridSearchCV`は先程の`cross_validate`（関数）とは異なりクラスです．
`GridSearchCV`のインスタンスを作成する際に，ハイパーパラメータチューニングを行いたいモデルとハイパーパラメータの探索範囲を与えます．
そして，作成した`GridSearchCV`のインスタンスの`fit`メソッドを実行すると，交差検証を行い最も良いハイパーパラメータを見つけてくれます．
`GridSearchCV`のインスタンスを作成する際に，最低限指定しなければならない引数を説明します．
 - `estimator`：探索・評価をしたいモデル（例：`LogisticRegression`）
 - `param_grid`：探索を行うハイパーパラメータとその範囲，辞書型（keyがハイパーパラメータの名前（文字列），valueが探索範囲（リスト））．
 - `scoring`：評価関数，文字列（例："roc_auc"）
 - `cv`：分割の数，整数．
 
詳しくは[GridSearchCVのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)を参照してください．
これもやはり実際は更に柔軟です．
注意が必要なのが，「最も良い」の意味です．
`GridSearchCV`では，`scoring`は「モデル（予測結果）の良さ」を表す関数であるとしています．
つまり，**スコアの最も高い**ハイパーパラメータを返します．
`GridSearchCV`では，`scoring`に文字列ではなく自身の作成した関数を渡すこともできますが，例えば二乗誤差のように「モデル（予測結果）の悪さ」を表す関数を渡すと最も悪いハイパーパラメータが返ってきます．

文章で説明してもわかりにくいと思いますので，とりあえず`solver`の探索範囲を`['newton-cg', 'lbfgs', 'sag']`，`C`の探索範囲を先ほどと同じ範囲として動かしてみます
（環境によっては`LineSearchWarning: The line search algorithm did not converge`のようなメッセージが表示されるかもしれませんが，今は気にせず進めてください）．

In [ ]:
from sklearn.model_selection import GridSearchCV

estimator = LogisticRegression(class_weight='balanced') # 探索するモデルを決める
param_grid = { # 探索するハイパーパラメータと範囲を決める
    "C": c_list,
    "solver": ['newton-cg', 'lbfgs', 'sag'],
} # fit_interceptなどはここでは指定されていないので，ずっと同じ値

cv = GridSearchCV(estimator=estimator, # estimator引数でどのモデルを使うかを指定
                  param_grid=param_grid, # param_grid引数で探索するハイパーパラメータと範囲を指定
                  scoring='roc_auc', cv=5) # scoringで評価指標を，cvで分割数を指定 
cv.fit(X_train, y_train) # 交差検証を実行

交差検証が終了すると，当然，その結果が知りたいです．
`fit`実行後の`GridSeachCV`は，
1. 交差検証による評価で最も良かったハイパーパラメータ：`best_params_`
2. 最も良かったハイパーパラメータを用いたときのスコア：`best_score_`
3. 最も良かったハイパーパラメータを用いたときのモデル：`best_estimator_`

を持っています．
最も良かったハイパーパラメータとその交差検証スコアを確認し，さらに`best_estimator_`で予測した結果を保存してみましょう．

In [ ]:
print(cv.best_params_) # 最も良かったハイパーパラメータを見てみる
print(cv.best_score_) # 最も良かったハイパーパラメータの時のスコアを見てみる．
y_pred_lr_gridcv = cv.best_estimator_.predict_proba(X_test)[:,1] # 最も良かったモデルで推定する．実はcv.predictでも良い
np.savetxt(X=y_pred_lr_gridcv, fname='y_pred_lr_gridcv.txt')
print(y_pred_lr_gridcv)

結果が出てきました．
上の探索範囲ではこのモデルが最も良いと考えられるので，この結果を投稿してみると良いかもしれません（しなくても良いです）．

## サポートベクターマシンの学習と予測

前回は，分類モデルの例としてロジスティック回帰について紹介しました．分類モデルには様々な種類があり，それぞれ構造や特徴が異なります．今回は，その他の分類モデルの例としてサポートベクターマシン（SVM）について紹介します．

サポートベクターマシンはカーネル法を用いたモデルの代表例であり，学習，予測の際にカーネル関数を自由に設定することができます．これにより，もともとのデータを別の空間に写像し，写像先の空間において分類する問題を考えることができます．単純な2値分類に対するサポートベクターマシンモデルは，ある関数（特徴写像）$\phi$ を用いた線形モデル

$$
 y(\mathbf{x}) = \mathbf{w}^{\top} \phi(\mathbf{x}) + b= \sum_{j=1}^{D}w_j\phi(x_j) + b.
$$

を変形することで，以下のように定式化されます．

$$
 \tilde y(\mathbf{x}) = \sum_{i=1}^N a_i t_i k(\mathbf{x},\mathbf{x}_i) + b
$$

ただし，$a_i$は$i$番目のデータに関するパラメータ，$t_i$は$i$番目のデータの正解の値，$x_i$は$i$番目のデータ中の正解の値以外のデータを指します．また，$x$は$y(x)$を予測したいデータであり，$b$はバイアス項を意味するパラメータです．

学習の流れとしては，大まかに分けると

 1. 訓練データから最適な$(a_1,\ldots,a_N)$の値を探す

 2. 求めた$(a_1,\ldots,a_N)$を用いて$\tilde y(\mathbf{x})$を計算

 3. $\tilde y(\mathbf{x})$の値から分類先を決定

のようになります．詳細については説明が長くなるため，ここでは省略します．気になる人は

 - 「パターン認識と機械学習　下」第7章（2002，C.M.Bishop，丸善出版）

 - 「サポートベクトルマシン（機械学習プロフェッショナルシリーズ）」（2015，竹内一郎・烏山昌幸，講談社）

などを読んでみてください．

それでは早速，モデルを動かしてみましょう．scikit-learnには`SVC`というモデルが用意されているので，これを使います．

In [ ]:
from sklearn.svm import SVC

svm = SVC(class_weight='balanced',probability=True) # クラスの重みをbalancedにし，確率で出力
svm.fit(X_train, y_train)
y_pred_svm = svm.predict_proba(X_test)[:,1] # クラス1に分類する確率の出力
np.savetxt(X=y_pred_svm, fname='y_pred_svm.txt')

In [ ]:
print(y_pred_svm)

この結果を提出してみると，私の環境では0.492124という非常に悪いスコアがでました．
ここで，sklearnのSVMのユーザーガイドから[Tips on Practical Use](https://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)の部分を見てみると，箇条書きの4個目に

"Support Vector Machine algorithms are not scale invariant, so it is highly recommended to scale your data. For example, scale each attribute on the input vector X to [0,1] or [-1,+1], or standardize it to have mean 0 and variance 1. Note that the same scaling must be applied to the test vector to obtain meaningful results. See section Preprocessing data for more details on scaling and normalization."

とあります．要約すると，

「データのスケーリングをすることを強く推奨する．例えば，区間[0,1]または[-1, 1]に入るようにする，もしくは平均0，分散1になるよう標準化する等．訓練データをスケーリングしたらテストデータもスケーリングするのを忘れずに．」

と言っています．スケーリングすると改善する可能性が見えてきました．今回は**標準化**をしてみます.

標準化では各特徴量の平均が0，分散が1になるようにデータに変換を施します．すなわち，訓練データを用いて平均ベクトル$\mathbb{\mu}\in\mathbb{R}^d$と標準偏差ベクトル$\mathbb{\sigma}\in\mathbb{R}^d$を計算して，
$$
  \mathbf{x'} = \frac{(\mathbf{x}-\mathbf{\mu})}{\mathbf{\sigma}}
$$
と定義する$\mathbf{x'}$を元の$\textbf{x}$
の代わりに用います（割り算は要素ごとに行う）．これにより，特徴ベクトルの各要素の平均が0分散が1になります．
幸運なことに，sklearnには`preprocessing.StandardScaler`として標準化のためのクラスが実装されています．動かしてみましょう．

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline #標準化の操作を1つのpipeにまとめる
std_svm = make_pipeline(StandardScaler(), SVC(class_weight='balanced',probability=True))
std_svm.fit(X_train, y_train) 
y_pred_std_svm = std_svm.predict_proba(X_test)[:,1]
np.savetxt(X=y_pred_std_svm, fname='y_pred_std_svm.txt')

サポートベクターマシンについても交差検証やグリッドサーチを行ってみると，スコアを改善できるかもしれません．
興味のある人は，ぜひやってみましょう．

（以下のセルを実行する際は、時間がかかるので注意してください．私の環境では，実行完了までに14分程度かかりました．）

In [ ]:
from sklearn.model_selection import GridSearchCV

estimator = make_pipeline(StandardScaler(), SVC(class_weight='balanced',probability=True)) # 探索するモデルを決める
param_grid = { # 探索するハイパーパラメータと範囲を決める
    "svc__C": [10**i for i in range(-3,3)],
    "svc__kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "svc__gamma": ['auto', 'scale']
} # fit_interceptなどはここでは指定されていないので，ずっと同じ値

#print(estimator.get_params().keys()) # 指定可能な探索パラメータを出力

cv_svm = GridSearchCV(estimator=estimator, # estimator引数でどのモデルを使うかを指定
                  param_grid=param_grid, # param_grid引数で探索するハイパーパラメータと範囲を指定
                  scoring='roc_auc', cv=5) # scoringで評価指標を，cvで分割数を指定 
cv_svm.fit(X_train, y_train) # 交差検証を実行

print(cv_svm.best_params_) # 最も良かったハイパーパラメータを見てみる
print(cv_svm.best_score_) # 最も良かったハイパーパラメータの時のスコアを見てみる．
y_pred_svm_cv = cv_svm.best_estimator_.predict_proba(X_test)[:,1] # 最も良かったモデルで推定する．実はcv.predictでも良い
np.savetxt(X=y_pred_svm_cv, fname='y_pred_svm_cv.txt')
print(y_pred_svm_cv)

## まとめ
- K分割交差検証：ラベル付きデータをK分割し，K-1個で学習，残りの一個で評価をK回繰り返し，手法を評価する．
- グリッドサーチ：それぞれのハイパーパラメータの探索範囲を定めて，全ての組合せを試して良いハイパーパラメータを決める．
- `GridSearchCV`を使うとハイパーパラメータチューニングが簡単にできる．

今回はロジスティック回帰やサポートベクターマシンを取り上げましたが，予測手法は他にも多数あります．
**非常に単純なモデル**は実装の手間が少なく，まず試すべきかとは思いますが，非線形でより複雑なモデルの方が良い性能であることが多々あります．
他の手法であっても，`fit`で学習し`predict`で予測することは変わらないですし，`GridSearchCV`の使い方も同じです（勿論，ハイパーパラメータの名前や有効な範囲がわかってないといけないため，ドキュメントを読む必要は間違いなくありますが）．
是非いろいろ試してみてください．
また，sklearn以外にも機械学習ライブラリはありますので，興味がある方は調べてみると良いかもしれません．
sklearnには様々な手法が実装されていますが，特定の手法に特化したライブラリが他にいくつもあります．

グリッドサーチではハイパーパラメータの候補点をそれぞれの探索範囲の直積から取ってきましたが，候補点をランダムに決める（＝グリッドになっていない）ランダムサーチ（`RandomizedSearchCV`）であったり（少数のハイパーパラメータの影響が強い場合に有効），ハイパーパラメータに関しても最適化する（＝もっと賢くハイパーパラメータを探す）方法も提案されています．
後者についてはsklearnにはありませんが，他にライブラリがあるので，興味がある方は調べてみると良いかもしれません（一回の学習が重たい手法では，少ない学習回数で良いハイパーパラメータを見つけたいため，ハイパーパラメータチューニングの手法が重要になってきます）．
もっとレイヤーが上がったところだと，「どの手法を使うか」「どのような特徴ベクトルを作るか」というところも上手くやる，**Automated Machine Learning（AutoML）**（そのままですね）というものもあり，AutoMLのライブラリもあったりします．
興味がある方はこちらも調べてみると面白いかもしれません．

## Answer

In [ ]:
c_list = [10**i for i in range(-3,3)]
results = []
for c in c_list:
    lr = LogisticRegression(C=c, class_weight='balanced')
    scores = cross_validate(lr, X_train, y_train, cv=5, 
                            scoring="roc_auc")
    results.append(np.mean(scores["test_score"]))
    print(np.mean(scores["test_score"]))

result_best = np.max(results)
c_best = c_list[np.argmax(results)]
print(f"最良のC：{c_best}　交差検証スコア：{result_best}")